In [41]:
from sqlalchemy import create_engine
import requests
import sys
import pymysql

In [42]:
URL = "dbbikes.cvwut6jnqsvn.us-east-1.rds.amazonaws.com"
PORT = "3306"
DB = "dbbikes"
USER = "admin"
PASSWORD = "password"

In [43]:
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{URL}:{PORT}/{DB}",echo = True)

sql ="""
CREATE TABLE IF NOT EXISTS station (
address VARCHAR(256),
banking int,
bike_stands int,
bonus int,
contract_name varchar(256),
name varchar(256),
number int primary key,
position_lat real,
position_lng real,
status varchar(256))"""

engine.execute(sql)

2023-03-11 10:27:39,081 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-03-11 10:27:39,081 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-11 10:27:39,272 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-03-11 10:27:39,272 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-11 10:27:39,365 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-03-11 10:27:39,365 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-11 10:27:39,561 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS station (
address VARCHAR(256),
banking int,
bike_stands int,
bonus int,
contract_name varchar(256),
name varchar(256),
number int primary key,
position_lat real,
position_lng real,
status varchar(256))
2023-03-11 10:27:39,561 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-11 10:27:39,664 INFO sqlalchemy.engine.Engine COMMIT


In [44]:
api_endpoint = "https://api.jcdecaux.com/vls/v1/stations"
params = {
    "contract": "dublin",
    "apiKey": "e58b454e0c21037a669b05e177dd6d3e910eb9fb"
}

try:
    data_bikes = requests.get(api_endpoint,params=params).json()

except requests.exceptions.RequestException as e:
    print(e)
    sys.exit(1)

[{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 15, 'available_bikes': 15, 'status': 'OPEN', 'last_update': 1678529883000}, {'number': 30, 'contract_name': 'dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 12, 'available_bikes': 8, 'status': 'OPEN', 'last_update': 1678530141000}, {'number': 54, 'contract_name': 'dublin', 'name': 'CLONMEL STREET', 'address': 'Clonmel Street', 'position': {'lat': 53.336021, 'lng': -6.26298}, 'banking': False, 'bonus': False, 'bike_stands': 33, 'available_bike_stands': 27, 'available_bikes': 6, 'status': 'OPEN', 'last_update': 1678530042000}, {'number': 108, 'contract_name': 'dublin', 'name': 'AVONDALE ROAD', 

In [45]:
sql_bikes = "INSERT IGNORE INTO station (address, banking, bike_stands, " \
            "bonus, contract_name, name, number, status, position_lat, position_lng) " \
            "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"


In [46]:
# Iterate through the data response object and perform inserts

for elem in range(len(data_bikes)):
    val = (data_bikes[elem]["address"], data_bikes[elem]["banking"], data_bikes[elem]["bike_stands"],
               data_bikes[elem]["bonus"], data_bikes[elem]["contract_name"], data_bikes[elem]["name"], data_bikes[elem]["number"], data_bikes[elem]["status"], data_bikes[elem]['position']['lat'], data_bikes[elem]['position']['lng'] )

    engine.execute(sql_bikes, val)


2023-03-11 10:27:40,170 INFO sqlalchemy.engine.Engine INSERT IGNORE INTO station (address, banking, bike_stands, bonus, contract_name, name, number, status, position_lat, position_lng) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-11 10:27:40,170 INFO sqlalchemy.engine.Engine [raw sql] ('Smithfield North', False, 30, False, 'dublin', 'SMITHFIELD NORTH', 42, 'OPEN', 53.349562, -6.278198)
2023-03-11 10:27:40,264 INFO sqlalchemy.engine.Engine COMMIT
2023-03-11 10:27:40,446 INFO sqlalchemy.engine.Engine INSERT IGNORE INTO station (address, banking, bike_stands, bonus, contract_name, name, number, status, position_lat, position_lng) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-11 10:27:40,446 INFO sqlalchemy.engine.Engine [raw sql] ('Parnell Square North', False, 20, False, 'dublin', 'PARNELL SQUARE NORTH', 30, 'OPEN', 53.3537415547453, -6.26530144781526)
2023-03-11 10:27:40,542 INFO sqlalchemy.engine.Engine COMMIT
2023-03-11 10:27:40,739 INFO sqlalchemy.engine.Engine I